In [23]:
# Importing the necessary libraries
import pandas as pd
from sqlalchemy import create_engine

In [25]:
validation_data = pd.read_csv('validation_data.csv')
validation_data.head()


,mmr,car_age,odometer_per_year,price_difference_from_mmr,condition_2.0,condition_3.0,condition_4.0,condition_5.0,condition_11.0,condition_12.0,...,make_44,make_45,make_46,make_47,make_48,make_49,make_50,make_51,make_52,log_sellingprice
0,0.122911,0.24,0.048449,0.480288,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.098
1,0.111954,0.04,0.009417,0.463506,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.863
2,0.126282,0.12,0.021509,0.477091,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.098
3,0.066442,0.08,0.030938,0.467768,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.384
4,0.100716,0.04,0.036594,0.464038,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.758


In [27]:
X = validation_data.drop('log_sellingprice', axis=1)
y = validation_data['log_sellingprice']

In [29]:
import joblib

model = joblib.load('random_forest_model.joblib')

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.6.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.6.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [31]:
y_pred = model.predict(X)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [33]:
# Checking the performance of the model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y, y_pred))
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Performance Metrics:")
print(f"RMSE: {rmse:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

Performance Metrics:
RMSE: 0.0116
MSE: 0.0001
R2 Score: 0.9998


The validation dataset performance indicates that the model is highly accurate, with an RMSE of 0.0123 and an MSE of 0.0002, suggesting very low prediction errors. The R² score of 0.9998 demonstrates that the model explains nearly all the variance in the validation data, confirming its strong predictive capability and effective generalization to unseen data.

In [35]:
validation_data['Selling Price Predicted'] = y_pred

In [43]:
# Saving the predictions to database
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'3637' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'analytics' # schema we just created

In [45]:
engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{db}')

In [47]:
validation_data.to_sql('validation_data', con=engine, if_exists='replace', index=False, schema=schema)

192

In [ ]:
validation_data.to_sql('validation_data', con=engine, if_exists='replace', index=False, chunksize=2000, method='multi', schema=schema)